In [1]:
import os
import shutil
import random

source_base_dir = '/kaggle/input/image-pro/Image_Processing'  


base_dir = '/kaggle/working/'


original_dir = os.path.join(source_base_dir, 'Original')
pixelated_dir = os.path.join(source_base_dir, 'Pixelated')


train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')


os.makedirs(os.path.join(train_dir, 'Original'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'Pixelated'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Original'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Pixelated'), exist_ok=True)


split_ratio = 0.8

def split_data(original_dir, pixelated_dir, train_dir, test_dir, split_ratio):
    original_files = os.listdir(original_dir)
    pixelated_files = os.listdir(pixelated_dir)

    
    paired_files = [(file, file) for file in original_files if file in pixelated_files]

    random.shuffle(paired_files)
    split_index = int(len(paired_files) * split_ratio)
    train_files = paired_files[:split_index]
    test_files = paired_files[split_index:]

    for original_file, pixelated_file in train_files:
        shutil.copy(os.path.join(original_dir, original_file), os.path.join(train_dir, 'Original', original_file))
        shutil.copy(os.path.join(pixelated_dir, pixelated_file), os.path.join(train_dir, 'Pixelated', pixelated_file))
    
    for original_file, pixelated_file in test_files:
        shutil.copy(os.path.join(original_dir, original_file), os.path.join(test_dir, 'Original', original_file))
        shutil.copy(os.path.join(pixelated_dir, pixelated_file), os.path.join(test_dir, 'Pixelated', pixelated_file))

split_data(original_dir, pixelated_dir, train_dir, test_dir, split_ratio)

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths to your dataset
train_dir = '/kaggle/working/train'
test_dir = '/kaggle/working/test'

# Data generators for training and validation/test
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Constants
input_shape = (227, 227, 3)
num_classes = 1
batch_size = 32
epochs = 10

2024-06-14 10:16:14.085416: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 10:16:14.085584: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 10:16:14.250327: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def SqueezeNet(input_shape=(227, 227, 3), num_classes=1):
    model = Sequential()

    # Layer 1: Convolution (96 filters, kernel size 7x7, stride 2)
    model.add(Conv2D(96, (7, 7), strides=(2, 2), activation='relu', input_shape=input_shape))

    # Layer 2: Max pooling (pool size 3x3, stride 2)
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Layer 3: Fire module 1
    model.add(fire_module(16, 64))

    # Layer 4: Fire module 2
    model.add(fire_module(16, 64))

    # Layer 5: Max pooling (pool size 3x3, stride 2)
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Layer 6: Fire module 3
    model.add(fire_module(32, 128))

    # Layer 7: Fire module 4
    model.add(fire_module(32, 128))

    # Layer 8: Max pooling (pool size 3x3, stride 2)
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Layer 9: Fire module 5
    model.add(fire_module(48, 192))

    # Layer 10: Fire module 6
    model.add(fire_module(48, 192))

    # Layer 11: Fire module 7
    model.add(fire_module(64, 256))

    # Layer 12: Fire module 8
    model.add(fire_module(64, 256))

    # Layer 13: Dropout (0.5)
    model.add(Dropout(0.5))

    # Layer 14: Convolution (num_classes filters, kernel size 1x1, activation softmax)
    model.add(Conv2D(num_classes, (1, 1), activation='sigmoid'))

    # Layer 15: Global average pooling
    model.add(GlobalAveragePooling2D())

    return model

def fire_module(squeeze, expand):
    return Sequential([
        Conv2D(squeeze, (1, 1), activation='relu'),
        Conv2D(expand, (3, 3), activation='relu', padding='same')
    ])

# Create the model
model = SqueezeNet(input_shape=input_shape, num_classes=num_classes)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 111, 111, 96)   │        14,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 55, 55, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 55, 55, 64)     │        10,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (None, 55, 55, 64)     │        10,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 27, 27, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (None, 27, 27, 128)    │        39,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ (None, 27, 27, 128)    │        41,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_5 (Sequential)       │ (None, 13, 13, 192)    │        89,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_6 (Sequential)       │ (None, 13, 13, 192)    │        92,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_7 (Sequential)       │ (None, 13, 13, 256)    │       160,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_8 (Sequential)       │ (None, 13, 13, 256)    │       164,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 13, 13, 1)      │           257 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1)              │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 621,761 (2.37 MB)

 Trainable params: 621,761 (2.37 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(227, 227),
    batch_size=batch_size,
    class_mode='binary'  # Binary labels (Pixelated vs Original)
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(227, 227),
    batch_size=batch_size,
    class_mode='binary'
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

Found 7884 images belonging to 2 classes.
Found 1972 images belonging to 2 classes.
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


246/246 ━━━━━━━━━━━━━━━━━━━━ 403s 2s/step - accuracy: 0.4949 - loss: 0.6932 - val_accuracy: 0.4995 - val_loss: 0.6931
Epoch 2/10
  1/246 ━━━━━━━━━━━━━━━━━━━━ 4:51 1s/step - accuracy: 0.5938 - loss: 0.6930

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 940us/step - accuracy: 0.5938 - loss: 0.6930 - val_accuracy: 0.5500 - val_loss: 0.6931
Epoch 3/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 386s 2s/step - accuracy: 0.4841 - loss: 0.6932 - val_accuracy: 0.4985 - val_loss: 0.6932
Epoch 4/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.4062 - loss: 0.6935 - val_accuracy: 0.6500 - val_loss: 0.6926
Epoch 5/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 399s 2s/step - accuracy: 0.5024 - loss: 0.6932 - val_accuracy: 0.5026 - val_loss: 0.6931
Epoch 6/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 820us/step - accuracy: 0.4375 - loss: 0.6936 - val_accuracy: 0.2500 - val_loss: 0.6949
Epoch 7/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 386s 2s/step - accuracy: 0.5039 - loss: 0.6931 - val_accuracy: 0.5005 - val_loss: 0.6931
Epoch 8/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 7s 25ms/step - accuracy: 0.4688 - loss: 0.6932 - val_accuracy: 0.4500 - val_loss: 0.6933
Epoch 9/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 405s 2s/step - accuracy: 0.4990 - loss: 0.6932 - val_accuracy: 0.4

In [5]:
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test accuracy: {accuracy:.4f}')

61/61 ━━━━━━━━━━━━━━━━━━━━ 35s 565ms/step - accuracy: 0.5044 - loss: 0.6931
Test accuracy: 0.4985


In [6]:
model.save('/kaggle/working/squeezenet_pixel_detect_model.h5')